In [1]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/Colab Notebooks')

Mounted at /content/drive


In [2]:
!pip install -q pyomo
!pip install pypsa
!apt-get install -y -qq glpk-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 70.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.6/158.6 kB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.0/354.0 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 74.5 MB/s eta 0:00:00
  Created wheel for pypsa: filename=pypsa-0.22.1-py2.py3-none-any.whl size=156349 sha256=52c76dc457ab285b9e6aa5d273d952fa49ed61e7eb48a0d5ea9d438e7e7097db
  Stored in directory: /root/.cache/pip/wheels/e6/35/b2/1e8791789e2f3f2ae90fd5ee0a1b10dd538a83ffa653ce5e6d
Successfully built pypsa
Selecting pr

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pypsa

In [4]:
df_genData=pd.read_excel(io="Genloads.xlsx",sheet_name="Sheet1",skiprows=0)
df_load=pd.read_excel(io="load.xlsx",sheet_name="Average",skiprows=0)

In [5]:
n = pypsa.Network()
years = [2030]
freq = "1" #Every hour

snapshots = pd.DatetimeIndex([])
for year in years:
    period = pd.date_range(
        start="{}-01-01 00:00".format(year),
        freq="{}H".format(freq),
        periods=8760 / float(freq),
    )
    snapshots = snapshots.append(period)

# convert to multiindex and assign to network
n.snapshots = pd.MultiIndex.from_arrays([snapshots.year, snapshots])
n.investment_periods = years

In [6]:
wind = df_genData['Wind'].squeeze()
wind = df_genData['Wind']
p_nom_wind = wind.set_axis(n.snapshots)      #This process attaches date/time to each of the gen loads
p_nom_wind

period  timestep           
2030    2030-01-01 00:00:00    0.017989
        2030-01-01 01:00:00    0.028384
        2030-01-01 02:00:00    0.028072
        2030-01-01 03:00:00    0.049640
        2030-01-01 04:00:00    0.069341
                                 ...   
        2030-12-31 19:00:00    0.535808
        2030-12-31 20:00:00    0.598553
        2030-12-31 21:00:00    0.598546
        2030-12-31 22:00:00    0.659696
        2030-12-31 23:00:00    0.734798
Name: Wind, Length: 8760, dtype: float64

In [7]:
gas = df_genData['Gas'].squeeze()
gas = df_genData['Gas']
p_nom_gas = gas.set_axis(n.snapshots)

In [8]:
coal = df_genData['Coal'].squeeze()
coal = df_genData['Coal']
p_nom_coal = coal.set_axis(n.snapshots)

In [9]:
solar = df_genData['Solar'].squeeze()
solar = df_genData['Solar']
p_nom_solar = solar.set_axis(n.snapshots)

In [10]:
Nuc_M = df_genData['Nuclear_Main'].squeeze()
Nuc_M = df_genData['Nuclear_Main']
p_nom_Nuclear_Main = Nuc_M.set_axis(n.snapshots)

In [11]:
Nuc_F = df_genData['Nuclear_Fix'].squeeze()
Nuc_F = df_genData['Nuclear_Fix']
p_nom_Nuclear_Fix = Nuc_F.set_axis(n.snapshots)

In [12]:
for i in range(3):
    n.add("Bus", "bus {}".format(i))

In [13]:
n.buses

attribute,v_nom,type,x,y,carrier,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,sub_network
Bus,,,,,,,,,,,
bus 0,1.0,,0.0,0.0,AC,None,1.0,0.0,inf,PQ,
bus 1,1.0,,0.0,0.0,AC,None,1.0,0.0,inf,PQ,
bus 2,1.0,,0.0,0.0,AC,None,1.0,0.0,inf,PQ,


In [14]:
n.add(
    "Line",
    "line 0-1",
    bus0="bus 0",
    bus1="bus 1",
    capital_cost= 20833,
)

n.add(
    "Line",
    "line 0-2",
    bus0="bus 0",
    bus1="bus 2",
    capital_cost= 20833,
)

n.add(
    "Line",
    "line 1-2",
    bus0="bus 1",
    bus1="bus 2",
    capital_cost= 20833,
)

n.lines["x"] = .001
n.lines["r"] = 0.02
n.lines["s_nom"] = 900
n.lines["s_nom_min"] = 900
n.lines["s_nom_extendable"] = True

#Estimated cost of extending line capacity by one MW = 250 mile long line = $1,125,000,000. Annualized over 60 years lifetime
#and for an expansion of up too another 900 MW, so divided by 900, = $20,833/MW expansion

In [15]:
n.lines

attribute,bus0,bus1,type,x,r,g,b,s_nom,s_nom_extendable,s_nom_min,...,v_ang_min,v_ang_max,sub_network,x_pu,r_pu,g_pu,b_pu,x_pu_eff,r_pu_eff,s_nom_opt
Line,,,,,,,,,,,,,,,,,,,,,
line 0-1,bus 0,bus 1,,0.001,0.02,0.0,0.0,900,True,900,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
line 0-2,bus 0,bus 2,,0.001,0.02,0.0,0.0,900,True,900,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0
line 1-2,bus 1,bus 2,,0.001,0.02,0.0,0.0,900,True,900,...,-inf,inf,,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# Generators, Nuclear_Main is for maintenance period, and Nuclear_Fixed for post-fix (different loads)

# n.add(
#     "Generator",
#     "A",
#     bus="bus 0",
#     p_nom=1148.4,
#     build_year=2024,
#     lifetime=10,
#     marginal_cost=44,
#     capital_cost=0,
#     p_max_pu=p_nom_coal,
#     carrier="coal",
#     p_nom_extendable=False,
# )

# n.add(
#     "Generator",
#     "B",
#     bus="bus 0",
#     p_nom=413.6,
#     build_year=2024,
#     lifetime=10,
#     marginal_cost=47,
#     capital_cost=0,
#     p_max_pu=p_nom_coal,
#     carrier="coal",
#     p_nom_extendable=False,
# )

n.add(
    "Generator",
    "C",
    bus="bus 1",
    p_nom=2491.2,
    build_year=2024,
    lifetime=10,
    marginal_cost=37,
    capital_cost=0,
    p_max_pu=p_nom_coal,
    carrier="coal",
    p_nom_extendable=False,
)

n.add(
    "Generator",
    "D",
    bus="bus 1",
    p_nom=1530.5,
    build_year=2024,
    lifetime=10,
    marginal_cost=35,
    capital_cost=0,
    p_max_pu=p_nom_coal,
    carrier="coal",
    p_nom_extendable=False,
)

# n.add(
#     "Generator",
#     "E",
#     bus="bus 2",
#     p_nom=2119,
#     build_year=2024,
#     lifetime=10,
#     marginal_cost=39,
#     capital_cost=0,
#     p_max_pu=p_nom_coal,
#     carrier="coal",
#     p_nom_extendable=False,
# )

# n.add(
#     "Generator",
#     "F",
#     bus="bus 2",
#     p_nom=763.2,
#     build_year=2024,
#     lifetime=10,
#     marginal_cost=45,
#     capital_cost=0,
#     p_max_pu=p_nom_coal,
#     carrier="coal",
#     p_nom_extendable=False,
# )


# n.add(
#     "Generator",
#     "G",
#     bus="bus 2",
#     p_nom=2558.2,
#     build_year=2024,
#     lifetime=10,
#     marginal_cost=38,
#     capital_cost=0,
#     p_max_pu=p_nom_coal,
#     carrier="coal",
#     p_nom_extendable=False,
# )

# n.add(
#     "Generator",
#     "H",
#     bus="bus 0",
#     p_nom=423.5,
#     build_year=2024,
#     lifetime=10,
#     marginal_cost=32,
#     capital_cost=0,
#     p_max_pu=p_nom_gas,
#     carrier="Gas",
#     p_nom_extendable=False,
# )

n.add(
    "Generator",
    "I",
    bus="bus 0",
    p_nom=697.9,
    build_year=2024,
    lifetime=10,
    marginal_cost=19,
    capital_cost=0,
    p_max_pu=p_nom_gas,
    carrier="Gas",
    p_nom_extendable=False,
)

n.add(
    "Generator",
    "J",
    bus="bus 0",
    p_nom=697.9,
    build_year=2024,
    lifetime=10,
    marginal_cost=22,
    capital_cost=0,
    p_max_pu=p_nom_gas,
    carrier="Gas",
    p_nom_extendable=False,
)

n.add(
    "Generator",
    "K",
    bus="bus 0",
    p_nom=583.2,
    build_year=2024,
    lifetime=10,
    marginal_cost=40,
    capital_cost=0,
    p_max_pu=p_nom_gas,
    carrier="Gas",
    p_nom_extendable=False,
)

n.add(
    "Generator",
    "L",
    bus="bus 0",
    p_nom=920,
    build_year=2024,
    lifetime=10,
    marginal_cost=23,
    capital_cost=0,
    p_max_pu=p_nom_gas,
    carrier="Gas",
    p_nom_extendable=False,
)

n.add(
    "Generator",
    "R",
    bus="bus 0",
    p_nom=271.2,
    build_year=2024,
    lifetime=10,
    marginal_cost=33,
    capital_cost=0,
    p_max_pu=p_nom_gas,
    carrier="Gas",
    p_nom_extendable=False,
)

# n.add(
#     "Generator",
#     "M",
#     bus="bus 1",
#     p_nom=1753.6,
#     build_year=2024,
#     lifetime=10,
#     marginal_cost=46,
#     capital_cost=0,
#     p_max_pu=p_nom_gas,
#     carrier="Gas",
#     p_nom_extendable=False,
# )

n.add(
    "Generator",
    "N",
    bus="bus 1",
    p_nom=799.2,
    build_year=2024,
    lifetime=10,
    marginal_cost=36,
    capital_cost=0,
    p_max_pu=p_nom_gas,
    carrier="Gas",
    p_nom_extendable=False,
)

n.add(
    "Generator",
    "O",
    bus="bus 1",
    p_nom=977.5,
    build_year=2024,
    lifetime=10,
    marginal_cost=30,
    capital_cost=0,
    p_max_pu=p_nom_gas,
    carrier="Gas",
    p_nom_extendable=False,
)

n.add(
    "Generator",
    "P",
    bus="bus 1",
    p_nom=2244.8,
    build_year=2024,
    lifetime=20,
    marginal_cost=24,
    capital_cost=0,
    p_max_pu=p_nom_gas,
    carrier="Gas",
    p_nom_extendable=False,
)

n.add(
    "Generator",
    "Q",
    bus="bus 2",
    p_nom=730,
    build_year=2024,
    lifetime=20,
    marginal_cost=21,
    capital_cost=0,
    p_max_pu=p_nom_gas,
    carrier="Gas",
    p_nom_extendable=False,
)

n.add(
    "Generator",
    "S",
    bus="bus 2",
    p_nom=979.7,
    build_year=2024,
    lifetime=10,
    marginal_cost=31,
    capital_cost=0,
    p_max_pu=p_nom_gas,
    carrier="Gas",
    p_nom_extendable=False,
)

# n.add(
#     "Generator",
#     "T",
#     bus="bus 2",
#     p_nom=979.7,
#     build_year=2024,
#     lifetime=10,
#     marginal_cost=25,
#     capital_cost=0,
#     p_max_pu=p_nom_gas,
#     carrier="Gas",
#     p_nom_extendable=False,
# )

n.add(
    "Generator",
    "N1_f",
    bus="bus 0",
    p_nom=2410,
    build_year=2024,
    lifetime=20,
    marginal_cost=12,
    capital_cost=0,
    p_max_pu=p_nom_Nuclear_Fix,
    carrier="Nuclear_Fix",
    p_nom_extendable=False,
)

n.add(
    "Generator",
    "N2_f",
    bus="bus 1",
    p_nom=2500,
    build_year=2024,
    lifetime=20,
    marginal_cost=13,
    capital_cost=0,
    p_max_pu=p_nom_Nuclear_Fix,
    carrier="Nuclear_Fix",
    p_nom_extendable=False,
)

n.add(
    "Generator",
    "N3_f",
    bus="bus 1",
    p_nom=2800,
    build_year=2024,
    lifetime=20,
    marginal_cost=14,
    capital_cost=0,
    p_max_pu=p_nom_Nuclear_Fix,
    carrier="Nuclear_Fix",
    p_nom_extendable=False,
)

n.add(
    "Generator",
    "N4_f",
    bus="bus 2",
    p_nom=2882,
    build_year=2024,
    lifetime=20,
    marginal_cost=15,
    capital_cost=0,
    p_max_pu=p_nom_Nuclear_Fix,
    carrier="Nuclear_Fix",
    p_nom_extendable=False,
)

n.add(
    "Generator",
    "N5_f",
    bus="bus 2",
    p_nom=768.6,
    build_year=2024,
    lifetime=20,
    marginal_cost=16,
    capital_cost=0,
    p_max_pu=p_nom_Nuclear_Fix,
    carrier="Nuclear_Fix",
    p_nom_extendable=False,
)

n.add(
    "Generator",
    "N6_f",
    bus="bus 2",
    p_nom=950.9,
    build_year=2024,
    lifetime=20,
    marginal_cost=17,
    capital_cost=0,
    p_max_pu=p_nom_Nuclear_Fix,
    carrier="Nuclear_Fix",
    p_nom_extendable=False,
)

n.add(
    "Generator",
    "IM1",
    bus="bus 0",
    p_nom=1250,
    build_year=2024,
    lifetime=20,
    marginal_cost=90,
    capital_cost=0,
    p_max_pu=p_nom_gas,
    carrier="Import",
    p_nom_extendable=False,
)

n.add(
    "Generator",
    "IM2",
    bus="bus 1",
    p_nom=1250,
    build_year=2024,
    lifetime=20,
    marginal_cost=100,
    capital_cost=0,
    p_max_pu=p_nom_gas,
    carrier="Import",
    p_nom_extendable=False,
)

n.add(
    "Generator",
    "S0",
    bus="bus 0",
    p_nom=1611,
    build_year=2024,
    lifetime=20,
    marginal_cost=0,
    capital_cost=0,
    p_max_pu=p_nom_solar,
    carrier="Solar",
    p_nom_extendable=False,
)

n.add(
    "Generator",
    "S1",
    bus="bus 1",
    p_nom=1611,
    build_year=2024,
    lifetime=20,
    marginal_cost=0, 
    capital_cost=0,
    p_max_pu=p_nom_solar,
    carrier="Solar",
    p_nom_extendable=False,
)

n.add(
    "Generator",
    "S2",
    bus="bus 2",
    p_nom=1611,
    build_year=2024,
    lifetime=20,
    marginal_cost=0,
    capital_cost=0,
    p_max_pu=p_nom_solar,
    carrier="Solar",
    p_nom_extendable=False,
)

n.add(
    "Generator",
    "S2_X",
    bus="bus 0",
    p_nom=0,
    build_year=2024,
    lifetime=20,
    marginal_cost=0,
    capital_cost=32770.03,
    p_max_pu=p_nom_solar,
    carrier="Solar",
    p_nom_extendable=True,
)

n.add(
    "Generator",
    "Wind",
    bus="bus 2",
    p_nom=0,
    build_year=2024,
    lifetime=20,
    marginal_cost=0,
    capital_cost=40283.33,    #Including $1,710 surcharge per MW for 62.13 mile transmissioat $4.5M a mile annualized over 30 years
    p_max_pu=p_nom_wind,
    carrier="Wind",
    p_nom_extendable=True, 
)

# n.add(
#     "StorageUnit",
#     "storageunit periodic",
#     bus="bus 2",
#     p_nom=0,
#     capital_cost=39100,
#     build_year=2024,
#     lifetime=21,
#     cyclic_state_of_charge=True,
#     cyclic_state_of_charge_per_period=True,
#     p_nom_extendable=True,
# )

# n.add(
#     "StorageUnit",
#     "storageunit periodic_two",
#     bus="bus 0",
#     p_nom=0,
#     capital_cost=39100,
#     build_year=2024,
#     lifetime=21,
#     cyclic_state_of_charge=True,
#     cyclic_state_of_charge_per_period=True,
#     p_nom_extendable=True,
# )

# n.add(
#     "StorageUnit",
#     "storageunit periodic_three",
#     bus="bus 1",
#     p_nom=0,
#     capital_cost=39100,
#     build_year=2024,
#     lifetime=21,
#     cyclic_state_of_charge=True,
#     cyclic_state_of_charge_per_period=True,
#     p_nom_extendable=True,
# )
    
n.generators

attribute,bus,control,type,p_nom,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,p_set,...,shut_down_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,
C,bus 1,PQ,,2491.2,False,0.0,inf,0.0,1.0,0.0,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
D,bus 1,PQ,,1530.5,False,0.0,inf,0.0,1.0,0.0,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
I,bus 0,PQ,,697.9,False,0.0,inf,0.0,1.0,0.0,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
J,bus 0,PQ,,697.9,False,0.0,inf,0.0,1.0,0.0,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
K,bus 0,PQ,,583.2,False,0.0,inf,0.0,1.0,0.0,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
L,bus 0,PQ,,920.0,False,0.0,inf,0.0,1.0,0.0,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
R,bus 0,PQ,,271.2,False,0.0,inf,0.0,1.0,0.0,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
N,bus 1,PQ,,799.2,False,0.0,inf,0.0,1.0,0.0,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
O,bus 1,PQ,,977.5,False,0.0,inf,0.0,1.0,0.0,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0


In [19]:
#Add Load
load_0 = df_load['Load_0_2030'].squeeze()
load_0 = df_load['Load_0_2030']
load_0 = load_0.set_axis(n.snapshots)

load_1 = df_load['Load_1_2030'].squeeze()
load_1 = df_load['Load_1_2030']
load_1 = load_1.set_axis(n.snapshots)

load_2 = df_load['Load_2_2030'].squeeze()
load_2 = df_load['Load_2_2030']
load_2 = load_2.set_axis(n.snapshots)

In [20]:
n.add("Load", "0", bus="bus 0", p_set=load_0)

n.add("Load", "1", bus="bus 1", p_set=load_1)

n.add("Load", "2", bus="bus 2", p_set=load_2)

In [21]:
n.loads_t.p_set

Load                             0         1        2
period timestep                                      
2030   2030-01-01 00:00:00  3969.6   8931.60  6946.80
       2030-01-01 01:00:00  3969.6   8931.60  6946.80
       2030-01-01 02:00:00  3969.6   8931.60  6946.80
       2030-01-01 03:00:00  3969.6   8931.60  6946.80
       2030-01-01 04:00:00  3969.6   8931.60  6946.80
...                            ...       ...      ...
       2030-12-31 19:00:00  4583.4  10312.65  8020.95
       2030-12-31 20:00:00  4583.4  10312.65  8020.95
       2030-12-31 21:00:00  4583.4  10312.65  8020.95
       2030-12-31 22:00:00  3969.6   8931.60  6946.80
       2030-12-31 23:00:00  3969.6   8931.60  6946.80

[8760 rows x 3 columns]

In [22]:
n

PyPSA Network
Components:
 - Bus: 3
 - Generator: 25
 - Line: 3
 - Load: 3
Snapshots: 8760

In [23]:
#Takes about 12 minutes to run depending on speed of your laptop
n.optimize()

Writing variables.: 100%|██████████| 5/5 [00:00<00:00, 13.26it/s]


GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --lp /tmp/linopy-problem-50vsll_s.lp --output /tmp/linopy-solve-t9suz0h4.sol
Reading problem data from '/tmp/linopy-problem-50vsll_s.lp'...
525605 rows, 245286 columns, 854084 non-zeros
3377016 lines were read
GLPK Simplex Optimizer, v4.65
525605 rows, 245286 columns, 854084 non-zeros
Preprocessing...
100719 rows, 201503 columns, 385416 non-zeros
Scaling...
 A: min|aij| =  8.000e-06  max|aij| =  1.000e+02  ratio =  1.250e+07
GM: min|aij| =  2.600e-01  max|aij| =  3.846e+00  ratio =  1.479e+01
EQ: min|aij| =  6.777e-02  max|aij| =  1.000e+00  ratio =  1.476e+01
Constructing initial basis...
Size of triangular part is 100719
      0: obj =   5.615165875e+09 inf =   2.154e+08 (26280)
    812: obj =   5.887267179e+09 inf =   2.091e+08 (25486)
   1977: obj =   5.921553639e+09 inf =   2.043e+08 (24328) 9
   3172: obj =   5.917709760e+09 inf =   1.968e+08 (23132) 3
   3863: obj =   5.874811613e+09 inf =   1.943e+08

('ok', 'optimal')

In [24]:
#Generate Generator outputs

gen_case2_2030 = n.generators_t.p
gen_case2_2030.to_excel('gen_case2_2030.xlsx', index = False)

In [25]:
#Generate Bus Prices

prices_case2_2030 = n.buses_t.marginal_price
prices_case2_2030.to_excel('prices_case2_2030.xlsx', index = False)

In [26]:
#Generate Storage Gen Data

# storage_BAU_one = n.storage_units_t.p
# storage_BAU_one.to_excel('storage_BAU_one.xlsx', index = False)

In [27]:
#Generate Line Data 

# lines_case2_2040 = n.lines_t.p
# lines_case2_2040.to_excel('lines_case2_2040.xlsx', index = False)

In [28]:
# n.lines_t

In [29]:
n.buses_t.marginal_price

Bus                         bus 0  bus 1  bus 2
period timestep                                
2030   2030-01-01 00:00:00    0.0    0.0    0.0
       2030-01-01 01:00:00    0.0    0.0    0.0
       2030-01-01 02:00:00    0.0    0.0    0.0
       2030-01-01 03:00:00    0.0    0.0    0.0
       2030-01-01 04:00:00    0.0    0.0    0.0
...                           ...    ...    ...
       2030-12-31 19:00:00    0.0    0.0    0.0
       2030-12-31 20:00:00    0.0    0.0    0.0
       2030-12-31 21:00:00    0.0    0.0    0.0
       2030-12-31 22:00:00    0.0    0.0    0.0
       2030-12-31 23:00:00    0.0    0.0    0.0

[8760 rows x 3 columns]

In [30]:
# n.storage_units_t

In [1]:
c = "Generator"

df = n.generators_t.p.sum(level=0).T
df.T.plot.bar(
    stacked=True,
    edgecolor="white",
    width=1,
    ylabel="Generation",
    xlabel="2030 Genration Mix",
    rot=0,
    figsize=(10, 12),
)

NameError: ignored